# AI Travel Planner


## Make a plan
#### Go somewhere nice.
I want to conversationally shape my ideal vacation plan like talking with a real travel expert. 
I don't know all the details up front, but will know it when I see it.

In [1]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens, 
    )
    return response.choices[0].message["content"]

## Implement a complex task with multiple prompts

### Extract relevant product and category names

In [3]:
delimiter = "####"
system_message = f"""
You will be provided with customer service queries. \
The customer service query will be delimited with \
{delimiter} characters.
Output a python list of objects, where each object has \
the following format:
    'category': <one of Countries>,
OR
    'products': <a list of products that must \
    be found in the allowed products below>

Where the categories and products must be found in \
the customer service query.
If a product is mentioned, it must be associated with \
the correct category in the allowed products list below.
If no products or categories are found, output an \
empty list.

Allowed products: 

Countries category:
Greece
Costa Rica
Israel
Spain
Finland
Sweden
Norway
Afghanistan



Only output the list of objects, with nothing else.
"""
user_message_1 = f"""
 tell me countries I can go that are LGBTQ friendly. \
  . \
  """
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message_1}{delimiter}"},  
] 
category_and_product_response_1 = get_completion_from_messages(messages)
print(category_and_product_response_1)

[{'category': 'Countries'}]


In [4]:
user_message_2 = f"""
i want to go to a country that is LGBTQ friendly."""
messages =  [  
{'role':'system',
 'content': system_message},    
{'role':'user',
 'content': f"{delimiter}{user_message_2}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

[{'category': 'Countries'}]


### Retrieve detailed product information for extracted products and categories

In [5]:
# product information
products = {
    "Finland": {
        "name": "Finland",
        "category": "Countries",
        "rating": 4.5,
        "features": ["LGBTQ friendly", "rice pudding", "cold weather", "Nordic", "scenic nature"],
        "description": "Finland has fantastic summers, and is a magical winter wonderland. It is known for its saunas, reindeer, and abundant nature and forests.",
        "average temperature": "50F"
    },
    "Spain": {
        "name": "Spain",
        "category": "Countries",
        "rating": 4.7,
        "features": ["LGBTQ friendly", "warm weather", "beaches", "tropical", "boat tours"],
        "description": "Relax at the beach, warm your toes in the sand, and swim in the beautiful Mediterranean sea. Spain has abundant palm trees.",
        "average temperature": "85F"
    },
    "Afghanistan": {
        "name": "Afghanistan",
        "category": "Countries",
        "rating": 3.0,
        "features": ["warm weather", "mountains"],
        "description": "Hike in the mountains and enjoy the sun.",
        "average temperature": "85F"
    }
}

In [6]:
def get_product_by_name(name):
    return products.get(name, None)

def get_products_by_category(category):
    return [product for product in products.values() if product["category"] == category]

In [7]:
print(get_product_by_name("Finland"))

{'name': 'Finland', 'category': 'Countries', 'rating': 4.5, 'features': ['LGBTQ friendly', 'rice pudding', 'cold weather', 'Nordic', 'scenic nature'], 'description': 'Finland has fantastic summers, and is a magical winter wonderland. It is known for its saunas, reindeer, and abundant nature and forests.', 'average temperature': '50F'}


In [8]:
print(get_products_by_category("Countries"))

[{'name': 'Finland', 'category': 'Countries', 'rating': 4.5, 'features': ['LGBTQ friendly', 'rice pudding', 'cold weather', 'Nordic', 'scenic nature'], 'description': 'Finland has fantastic summers, and is a magical winter wonderland. It is known for its saunas, reindeer, and abundant nature and forests.', 'average temperature': '50F'}, {'name': 'Spain', 'category': 'Countries', 'rating': 4.7, 'features': ['LGBTQ friendly', 'warm weather', 'beaches', 'tropical', 'boat tours'], 'description': 'Relax at the beach, warm your toes in the sand, and swim in the beautiful Mediterranean sea. Spain has abundant palm trees.', 'average temperature': '85F'}, {'name': 'Afghanistan', 'category': 'Countries', 'rating': 3.0, 'features': ['warm weather', 'mountains'], 'description': 'Hike in the mountains and enjoy the sun.', 'average temperature': '85F'}]


In [9]:
print(user_message_1)


 tell me countries I can go that are LGBTQ friendly.   .   


In [10]:
print(category_and_product_response_1)

[{'category': 'Countries'}]


### Read Python string into Python list of dictionaries

In [11]:
import json 

def read_string_to_list(input_string):
    if input_string is None:
        return None

    try:
        input_string = input_string.replace("'", "\"")  # Replace single quotes with double quotes for valid JSON
        data = json.loads(input_string)
        return data
    except json.JSONDecodeError:
        print("Error: Invalid JSON string")
        return None   
    

In [12]:
category_and_product_list = read_string_to_list(category_and_product_response_1)
print(category_and_product_list)

[{'category': 'Countries'}]


#### Retrieve detailed product information for the relevant products and categories

In [13]:
def generate_output_string(data_list):
    output_string = ""

    if data_list is None:
        return output_string

    for data in data_list:
        try:
            if "products" in data:
                products_list = data["products"]
                for product_name in products_list:
                    product = get_product_by_name(product_name)
                    if product:
                        output_string += json.dumps(product, indent=4) + "\n"
                    else:
                        print(f"Error: Product '{product_name}' not found")
            elif "category" in data:
                category_name = data["category"]
                category_products = get_products_by_category(category_name)
                for product in category_products:
                    output_string += json.dumps(product, indent=4) + "\n"
            else:
                print("Error: Invalid object format")
        except Exception as e:
            print(f"Error: {e}")

    return output_string 

In [14]:
product_information_for_user_message_1 = generate_output_string(category_and_product_list)
print(product_information_for_user_message_1)

{
    "name": "Finland",
    "category": "Countries",
    "rating": 4.5,
    "features": [
        "LGBTQ friendly",
        "rice pudding",
        "cold weather",
        "Nordic",
        "scenic nature"
    ],
    "description": "Finland has fantastic summers, and is a magical winter wonderland. It is known for its saunas, reindeer, and abundant nature and forests.",
    "average temperature": "50F"
}
{
    "name": "Spain",
    "category": "Countries",
    "rating": 4.7,
    "features": [
        "LGBTQ friendly",
        "warm weather",
        "beaches",
        "tropical",
        "boat tours"
    ],
    "description": "Relax at the beach, warm your toes in the sand, and swim in the beautiful Mediterranean sea. Spain has abundant palm trees.",
    "average temperature": "85F"
}
{
    "name": "Afghanistan",
    "category": "Countries",
    "rating": 3.0,
    "features": [
        "warm weather",
        "mountains"
    ],
    "description": "Hike in the mountains and enjoy the su

### Generate answer to user query based on detailed product information

In [17]:
system_message = f"""
You are a customer service assistant for a travel agency. \
Respond in a friendly and helpful tone, \
with very concise answers. \
Make sure to ask the user relevant follow up questions.\
You should only use data in the products dictionary and nothing else. \
You should not list other products if it is not a match or not true. \
All criteria in the question must be met in your response, otherwise respond with an apology.\
If a criteria is not met, then say so, but do not offer alternatives.\
"""
# Note: 'Do not list other products if it is not a match.' 
user_message_1 = f"""
where can I visit that is LGBTQ friendly, has palm trees, and has boat tours? \
"""
messages =  [  
{'role':'system',
 'content': system_message},   
{'role':'user',
 'content': user_message_1},  
{'role':'assistant',
 'content': f"""Relevant product information:\n\
 {product_information_for_user_message_1}"""},   
]
final_response = get_completion_from_messages(messages)
print(final_response)

Spain would be a great destination for you! It is LGBTQ friendly, has palm trees, and offers boat tours.
